# Homework and bake-off: Sentiment analysis

## Set-up


In [1]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch import Tensor
import io
import time

from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to /Users/sunmilee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load Data


### Vocab

In [2]:
eng_train_url = 'train.en.gz'
eng_val_url = 'val.en.gz'
eng_test_url = 'test_2016_flickr.en.gz'
eng_spacy_lng = 'en_core_web_sm'

dst_train_url = 'train.fr.gz'
dst_val_url = 'val.fr.gz'
dst_test_url = 'test_2016_flickr.fr.gz'
dst_spacy_lng = 'fr_core_news_sm'

In [3]:
from torchtext.datasets import AG_NEWS, YelpReviewFull

url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = (eng_train_url, dst_train_url)
val_urls = (eng_val_url, dst_val_url)
test_urls = (eng_test_url, dst_test_url)

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

eng_lng_tokenizer = get_tokenizer('spacy', language=eng_spacy_lng)
dst_lng_tokenizer = get_tokenizer('spacy', language=dst_spacy_lng)
# classification_tokenizer = get_tokenizer('basic_english')

# Torchtext dataset
ag_train_iter, ag_test_iter = AG_NEWS()
ag_train_list = list(ag_train_iter)
ag_test_list = list(ag_test_iter)
ag_train_list_sub = ag_train_list[:int(len(ag_train_list) / 24)]
ag_test_list_sub = ag_test_list[:int(len(ag_test_list) / 15)]
yelp_train_iter, yelp_test_iter = YelpReviewFull()
yelp_train_list = list(yelp_train_iter)
yelp_test_list = list(yelp_test_iter)
yelp_train_list_sub = yelp_train_list[:int(len(yelp_train_list) / 130)]
yelp_test_list_sub = yelp_test_list[:int(len(yelp_test_list) / 15)]

print("AG_NEWS Train size:", len(ag_train_list))
print("AG_NEWS Test size:", len(ag_test_list))
print("YELP FULL Train size:", len(yelp_train_list))
print("YELP FULL Test size:", len(ag_test_list))

AG_NEWS Train size: 120000
AG_NEWS Test size: 7600
YELP FULL Train size: 650000
YELP FULL Test size: 7600


Destination language vocab

In [4]:
counter = Counter()
with io.open(train_filepaths[1], encoding="utf8") as f:
    for string_ in f:
        counter.update(dst_lng_tokenizer(string_))
dst_lng_vocab = Vocab(counter, min_freq=1, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

DST_VOCAB_SIZE = len(dst_lng_vocab)
print("Dest Vocab size:", DST_VOCAB_SIZE)

Dest Vocab size: 11510


English language vocab

In [5]:
counter = Counter()
with io.open(train_filepaths[0], encoding="utf8") as f:
    for string_ in f:
        counter.update(eng_lng_tokenizer(string_))
for i in range(len(ag_train_list)):
    counter.update(eng_lng_tokenizer(ag_train_list[i][1]))
for i in range(len(yelp_train_list)):
    counter.update(eng_lng_tokenizer(yelp_train_list[i][1]))

eng_lng_vocab = Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

ENG_VOCAB_SIZE = len(eng_lng_vocab)
print("English Vocab size:", ENG_VOCAB_SIZE)

English Vocab size: 926692


### Index

In [6]:
PAD_IDX = eng_lng_vocab['<pad>']
BOS_IDX = eng_lng_vocab['<bos>']
EOS_IDX = eng_lng_vocab['<eos>']
print(PAD_IDX)
print(BOS_IDX)
print(EOS_IDX)

1
2
3


## Translation Model

### Process Data

In [7]:
# Init tokens
def data_process(filepaths):
    raw_eng_lng_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_dst_lng_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_eng, raw_dst) in zip(raw_eng_lng_iter, raw_dst_lng_iter):
        eng_lng_tensor_ = torch.tensor([eng_lng_vocab[token] for token in eng_lng_tokenizer(raw_eng.rstrip("\n"))],
                                dtype=torch.long)
        dst_lng_tensor_ = torch.tensor([dst_lng_vocab[token] for token in dst_lng_tokenizer(raw_dst.rstrip("\n"))],
                                dtype=torch.long)
        data.append((eng_lng_tensor_, dst_lng_tensor_))
    return data


train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

### Load Data

In [8]:
from torch.nn.utils.rnn import pad_sequence

BATCH_SIZE = 128

def generate_batch(data_batch):
    dst_lng_batch, src_lng_batch = [], []
    for (src_lng_item, dst_lng_item) in data_batch:
        dst_lng_batch.append(torch.cat([torch.tensor([BOS_IDX]), dst_lng_item, torch.tensor([EOS_IDX])], dim=0))
        src_lng_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_lng_item, torch.tensor([EOS_IDX])], dim=0))
    dst_lng_batch = pad_sequence(dst_lng_batch, padding_value=PAD_IDX)
    src_lng_batch = pad_sequence(src_lng_batch, padding_value=PAD_IDX)
    return src_lng_batch, dst_lng_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

print("Train count:", len(train_iter.dataset))

Train count: 29000


See if data looks right

In [9]:
for idx, (src, tgt) in enumerate(train_iter):
    src = src.to(device)
    tgt = tgt.to(device)
    print(src)
    print(tgt)
    break

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 173, 2165, 2029,  ...,  173,  173,  173],
        [ 352, 2667, 2508,  ...,  514,  877,  213],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  8,  17,  71,  ...,   8,  17,   8],
        [187, 177,  39,  ...,  14,  32,  73],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])


### Transformer Model

Transformer

In [10]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

Position Encoder

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

We create a subsequent word mask to stop a target word from attending to its subsequent words. We also create masks, for masking source and target padding tokens

In [12]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

### Train

Define model parameters and instantiate model

In [13]:
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 25


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, ENG_VOCAB_SIZE, DST_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [14]:
def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()

        losses += loss.item()
    return losses / len(train_iter)


def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                  src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)

Train

In [15]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch(transformer, train_iter, optimizer)
    end_time = time.time()
#     val_loss = evaluate(transformer, valid_iter)
    val_loss = 0
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 5.379, Val loss: 0.000, Epoch time = 1123.197s
Epoch: 2, Train loss: 3.512, Val loss: 0.000, Epoch time = 1120.894s
Epoch: 3, Train loss: 2.724, Val loss: 0.000, Epoch time = 1125.906s
Epoch: 4, Train loss: 2.236, Val loss: 0.000, Epoch time = 1129.308s
Epoch: 5, Train loss: 1.902, Val loss: 0.000, Epoch time = 1119.605s
Epoch: 6, Train loss: 1.661, Val loss: 0.000, Epoch time = 1123.406s
Epoch: 7, Train loss: 1.474, Val loss: 0.000, Epoch time = 1117.327s
Epoch: 8, Train loss: 1.327, Val loss: 0.000, Epoch time = 1130.303s
Epoch: 9, Train loss: 1.201, Val loss: 0.000, Epoch time = 1126.160s
Epoch: 10, Train loss: 1.101, Val loss: 0.000, Epoch time = 1133.951s
Epoch: 11, Train loss: 1.011, Val loss: 0.000, Epoch time = 1124.575s
Epoch: 12, Train loss: 0.933, Val loss: 0.000, Epoch time = 1127.162s
Epoch: 13, Train loss: 0.865, Val loss: 0.000, Epoch time = 1130.115s
Epoch: 14, Train loss: 0.804, Val loss: 0.000, Epoch time = 1123.642s
Epoch: 15, Train loss: 0.748,

The models trained using transformer architecture — train faster and converge to a lower validation loss compared to RNN models.

### Translation Model In-Action

In [16]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [17]:
translate(transformer, "Hello", eng_lng_vocab, dst_lng_vocab, eng_lng_tokenizer)

' Des sauveteurs '

### Save & Load Model

Save model

In [18]:
PATH = './translation_model_eng_to_fr2.pt'

torch.save(transformer, PATH)

Load saved model

In [19]:
PATH = './translation_model_eng_to_fr2.pt'

transformer_model = torch.load(PATH)

In [20]:
translate(transformer_model, "My name is John", eng_lng_vocab, dst_lng_vocab, eng_lng_tokenizer)

' Oh semble apprécier son porte-voix John '

## Text Style Classification

### Data Processing

In [21]:
# Re-label
new_ag_train_list = []
new_ag_test_list = []
new_yelp_train_list = []
new_yelp_test_list = []
for i in range(len(ag_train_list)):
    new_ag_train_list.append((1, ag_train_list[i][1]))
for i in range(len(ag_test_list)):
    new_ag_test_list.append((1, ag_test_list[i][1]))
for i in range(len(yelp_train_list)):
    new_yelp_train_list.append((2, yelp_train_list[i][1]))
for i in range(len(yelp_test_list)):
    new_yelp_test_list.append((2, yelp_test_list[i][1]))
    
train_dataset = new_ag_train_list + new_yelp_train_list
test_dataset = new_ag_test_list + new_yelp_test_list

In [22]:
BATCH_SIZE = 2 # batch size for training

def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        text_list.append(torch.cat([torch.tensor([BOS_IDX]), 
                                    torch.tensor(text_pipeline(_text), dtype=torch.int64), 
                                    torch.tensor([EOS_IDX])], dim=0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = torch.transpose(pad_sequence(text_list, padding_value=PAD_IDX), 0, 1)
    return label_list.to(device), text_list.to(device)

num_class = 2
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

print("Train size:", len(train_dataloader.dataset))
print("Val size:", len(valid_dataloader.dataset))
print("Test size:", len(test_dataloader.dataset))

Train size: 731500
Val size: 38500
Test size: 57600


In [23]:
text_pipeline = lambda x: [eng_lng_vocab[token] for token in eng_lng_tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [24]:
text_pipeline('here is the an example')

[55, 17, 5, 68, 2220]

In [25]:
label_pipeline(10)

9

### Model

In [26]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    
    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)

In [27]:
emsize = 64
classification_model = TextClassificationModel(ENG_VOCAB_SIZE, emsize, num_class).to(device)

### Train

In [28]:
import time

def train(dataloader):
    classification_model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = classification_model(text)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(classification_model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    classification_model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predited_label = classification_model(text)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [29]:
temp_data = train_dataset[:2]
temp_dataloader = DataLoader(temp_data, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [30]:
# Hyperparameters
EPOCHS = 5 # epoch
LR = 1  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/365750 batches | accuracy    0.827
| epoch   1 |  1000/365750 batches | accuracy    0.854
| epoch   1 |  1500/365750 batches | accuracy    0.855
| epoch   1 |  2000/365750 batches | accuracy    0.831
| epoch   1 |  2500/365750 batches | accuracy    0.849
| epoch   1 |  3000/365750 batches | accuracy    0.850
| epoch   1 |  3500/365750 batches | accuracy    0.831
| epoch   1 |  4000/365750 batches | accuracy    0.867
| epoch   1 |  4500/365750 batches | accuracy    0.858
| epoch   1 |  5000/365750 batches | accuracy    0.884
| epoch   1 |  5500/365750 batches | accuracy    0.912
| epoch   1 |  6000/365750 batches | accuracy    0.909
| epoch   1 |  6500/365750 batches | accuracy    0.896
| epoch   1 |  7000/365750 batches | accuracy    0.913
| epoch   1 |  7500/365750 batches | accuracy    0.920
| epoch   1 |  8000/365750 batches | accuracy    0.916
| epoch   1 |  8500/365750 batches | accuracy    0.920
| epoch   1 |  9000/365750 batches | accuracy    0.921
| epoch   

| epoch   1 | 75000/365750 batches | accuracy    0.967
| epoch   1 | 75500/365750 batches | accuracy    0.962
| epoch   1 | 76000/365750 batches | accuracy    0.959
| epoch   1 | 76500/365750 batches | accuracy    0.972
| epoch   1 | 77000/365750 batches | accuracy    0.963
| epoch   1 | 77500/365750 batches | accuracy    0.969
| epoch   1 | 78000/365750 batches | accuracy    0.973
| epoch   1 | 78500/365750 batches | accuracy    0.963
| epoch   1 | 79000/365750 batches | accuracy    0.967
| epoch   1 | 79500/365750 batches | accuracy    0.961
| epoch   1 | 80000/365750 batches | accuracy    0.969
| epoch   1 | 80500/365750 batches | accuracy    0.965
| epoch   1 | 81000/365750 batches | accuracy    0.964
| epoch   1 | 81500/365750 batches | accuracy    0.971
| epoch   1 | 82000/365750 batches | accuracy    0.966
| epoch   1 | 82500/365750 batches | accuracy    0.974
| epoch   1 | 83000/365750 batches | accuracy    0.968
| epoch   1 | 83500/365750 batches | accuracy    0.969
| epoch   

| epoch   1 | 149000/365750 batches | accuracy    0.970
| epoch   1 | 149500/365750 batches | accuracy    0.967
| epoch   1 | 150000/365750 batches | accuracy    0.970
| epoch   1 | 150500/365750 batches | accuracy    0.977
| epoch   1 | 151000/365750 batches | accuracy    0.972
| epoch   1 | 151500/365750 batches | accuracy    0.977
| epoch   1 | 152000/365750 batches | accuracy    0.961
| epoch   1 | 152500/365750 batches | accuracy    0.973
| epoch   1 | 153000/365750 batches | accuracy    0.973
| epoch   1 | 153500/365750 batches | accuracy    0.967
| epoch   1 | 154000/365750 batches | accuracy    0.968
| epoch   1 | 154500/365750 batches | accuracy    0.973
| epoch   1 | 155000/365750 batches | accuracy    0.972
| epoch   1 | 155500/365750 batches | accuracy    0.962
| epoch   1 | 156000/365750 batches | accuracy    0.970
| epoch   1 | 156500/365750 batches | accuracy    0.975
| epoch   1 | 157000/365750 batches | accuracy    0.978
| epoch   1 | 157500/365750 batches | accuracy  

| epoch   1 | 222500/365750 batches | accuracy    0.981
| epoch   1 | 223000/365750 batches | accuracy    0.975
| epoch   1 | 223500/365750 batches | accuracy    0.974
| epoch   1 | 224000/365750 batches | accuracy    0.972
| epoch   1 | 224500/365750 batches | accuracy    0.976
| epoch   1 | 225000/365750 batches | accuracy    0.978
| epoch   1 | 225500/365750 batches | accuracy    0.983
| epoch   1 | 226000/365750 batches | accuracy    0.976
| epoch   1 | 226500/365750 batches | accuracy    0.978
| epoch   1 | 227000/365750 batches | accuracy    0.978
| epoch   1 | 227500/365750 batches | accuracy    0.977
| epoch   1 | 228000/365750 batches | accuracy    0.981
| epoch   1 | 228500/365750 batches | accuracy    0.978
| epoch   1 | 229000/365750 batches | accuracy    0.969
| epoch   1 | 229500/365750 batches | accuracy    0.975
| epoch   1 | 230000/365750 batches | accuracy    0.977
| epoch   1 | 230500/365750 batches | accuracy    0.976
| epoch   1 | 231000/365750 batches | accuracy  

| epoch   1 | 296000/365750 batches | accuracy    0.979
| epoch   1 | 296500/365750 batches | accuracy    0.981
| epoch   1 | 297000/365750 batches | accuracy    0.983
| epoch   1 | 297500/365750 batches | accuracy    0.986
| epoch   1 | 298000/365750 batches | accuracy    0.979
| epoch   1 | 298500/365750 batches | accuracy    0.980
| epoch   1 | 299000/365750 batches | accuracy    0.978
| epoch   1 | 299500/365750 batches | accuracy    0.971
| epoch   1 | 300000/365750 batches | accuracy    0.981
| epoch   1 | 300500/365750 batches | accuracy    0.983
| epoch   1 | 301000/365750 batches | accuracy    0.972
| epoch   1 | 301500/365750 batches | accuracy    0.987
| epoch   1 | 302000/365750 batches | accuracy    0.977
| epoch   1 | 302500/365750 batches | accuracy    0.978
| epoch   1 | 303000/365750 batches | accuracy    0.988
| epoch   1 | 303500/365750 batches | accuracy    0.979
| epoch   1 | 304000/365750 batches | accuracy    0.984
| epoch   1 | 304500/365750 batches | accuracy  

...

| epoch   5 |  8000/365750 batches | accuracy    0.993
| epoch   5 |  8500/365750 batches | accuracy    0.994
| epoch   5 |  9000/365750 batches | accuracy    0.996
| epoch   5 |  9500/365750 batches | accuracy    0.994
| epoch   5 | 10000/365750 batches | accuracy    0.999
| epoch   5 | 10500/365750 batches | accuracy    0.991
| epoch   5 | 11000/365750 batches | accuracy    0.992
| epoch   5 | 11500/365750 batches | accuracy    0.990
| epoch   5 | 12000/365750 batches | accuracy    0.995
| epoch   5 | 12500/365750 batches | accuracy    0.996
| epoch   5 | 13000/365750 batches | accuracy    0.994
| epoch   5 | 13500/365750 batches | accuracy    0.996
| epoch   5 | 14000/365750 batches | accuracy    0.991
| epoch   5 | 14500/365750 batches | accuracy    0.997
| epoch   5 | 15000/365750 batches | accuracy    0.997
| epoch   5 | 15500/365750 batches | accuracy    0.990
| epoch   5 | 16000/365750 batches | accuracy    0.993
| epoch   5 | 16500/365750 batches | accuracy    0.994
| epoch   

| epoch   5 | 82500/365750 batches | accuracy    0.998
| epoch   5 | 83000/365750 batches | accuracy    0.995
| epoch   5 | 83500/365750 batches | accuracy    0.998
| epoch   5 | 84000/365750 batches | accuracy    0.993
| epoch   5 | 84500/365750 batches | accuracy    0.996
| epoch   5 | 85000/365750 batches | accuracy    0.996
| epoch   5 | 85500/365750 batches | accuracy    0.995
| epoch   5 | 86000/365750 batches | accuracy    0.994
| epoch   5 | 86500/365750 batches | accuracy    0.998
| epoch   5 | 87000/365750 batches | accuracy    0.996
| epoch   5 | 87500/365750 batches | accuracy    0.997
| epoch   5 | 88000/365750 batches | accuracy    0.995
| epoch   5 | 88500/365750 batches | accuracy    0.995
| epoch   5 | 89000/365750 batches | accuracy    0.996
| epoch   5 | 89500/365750 batches | accuracy    0.998
| epoch   5 | 90000/365750 batches | accuracy    0.990
| epoch   5 | 90500/365750 batches | accuracy    0.997
| epoch   5 | 91000/365750 batches | accuracy    0.993
| epoch   

| epoch   5 | 156000/365750 batches | accuracy    0.994
| epoch   5 | 156500/365750 batches | accuracy    0.992
| epoch   5 | 157000/365750 batches | accuracy    0.995
| epoch   5 | 157500/365750 batches | accuracy    0.992
| epoch   5 | 158000/365750 batches | accuracy    0.994
| epoch   5 | 158500/365750 batches | accuracy    0.997
| epoch   5 | 159000/365750 batches | accuracy    0.994
| epoch   5 | 159500/365750 batches | accuracy    0.996
| epoch   5 | 160000/365750 batches | accuracy    0.993
| epoch   5 | 160500/365750 batches | accuracy    0.992
| epoch   5 | 161000/365750 batches | accuracy    0.989
| epoch   5 | 161500/365750 batches | accuracy    0.993
| epoch   5 | 162000/365750 batches | accuracy    0.998
| epoch   5 | 162500/365750 batches | accuracy    0.993
| epoch   5 | 163000/365750 batches | accuracy    0.994
| epoch   5 | 163500/365750 batches | accuracy    0.995
| epoch   5 | 164000/365750 batches | accuracy    0.995
| epoch   5 | 164500/365750 batches | accuracy  

| epoch   5 | 229500/365750 batches | accuracy    0.995
| epoch   5 | 230000/365750 batches | accuracy    0.996
| epoch   5 | 230500/365750 batches | accuracy    0.995
| epoch   5 | 231000/365750 batches | accuracy    0.995
| epoch   5 | 231500/365750 batches | accuracy    0.993
| epoch   5 | 232000/365750 batches | accuracy    0.993
| epoch   5 | 232500/365750 batches | accuracy    0.997
| epoch   5 | 233000/365750 batches | accuracy    0.999
| epoch   5 | 233500/365750 batches | accuracy    0.991
| epoch   5 | 234000/365750 batches | accuracy    0.994
| epoch   5 | 234500/365750 batches | accuracy    0.996
| epoch   5 | 235000/365750 batches | accuracy    0.995
| epoch   5 | 235500/365750 batches | accuracy    0.989
| epoch   5 | 236000/365750 batches | accuracy    0.996
| epoch   5 | 236500/365750 batches | accuracy    0.997
| epoch   5 | 237000/365750 batches | accuracy    0.998
| epoch   5 | 237500/365750 batches | accuracy    0.994
| epoch   5 | 238000/365750 batches | accuracy  

| epoch   5 | 303000/365750 batches | accuracy    0.999
| epoch   5 | 303500/365750 batches | accuracy    0.994
| epoch   5 | 304000/365750 batches | accuracy    0.996
| epoch   5 | 304500/365750 batches | accuracy    0.996
| epoch   5 | 305000/365750 batches | accuracy    0.996
| epoch   5 | 305500/365750 batches | accuracy    0.991
| epoch   5 | 306000/365750 batches | accuracy    0.994
| epoch   5 | 306500/365750 batches | accuracy    0.996
| epoch   5 | 307000/365750 batches | accuracy    0.990
| epoch   5 | 307500/365750 batches | accuracy    0.996
| epoch   5 | 308000/365750 batches | accuracy    0.996
| epoch   5 | 308500/365750 batches | accuracy    0.992
| epoch   5 | 309000/365750 batches | accuracy    0.993
| epoch   5 | 309500/365750 batches | accuracy    0.993
| epoch   5 | 310000/365750 batches | accuracy    0.993
| epoch   5 | 310500/365750 batches | accuracy    0.998
| epoch   5 | 311000/365750 batches | accuracy    0.996
| epoch   5 | 311500/365750 batches | accuracy  

### Test

In [31]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.995


In [32]:
style_label = {1: "AG_NEWS",
               2: "YELP"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.cat([torch.tensor([BOS_IDX]), 
                                    torch.tensor(text_pipeline(text), dtype=torch.int64), 
                                    torch.tensor([EOS_IDX])], dim=0)
        text = torch.reshape(text, (1, text.shape[0]))
        output = classification_model(text)
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

ex_text_str = "I think food is not so bad."

classification_model = classification_model.to("cpu")
classification_model.eval()

print("This is a %s style" %style_label[predict(ex_text_str, text_pipeline)])

PATH_CLS = './classification_model.pt'

torch.save(classification_model, PATH_CLS)

This is a YELP style


In [33]:
# classification_model = torch.load(PATH)

## Text Style Generator

In [34]:
# Tokenize & Re-label
train_data = []
test_data = []
for i in range(len(ag_train_list_sub)):
    sentences = sent_tokenize(ag_train_list_sub[i][1])
    for sentence in sentences:
        text_tensor = torch.tensor([eng_lng_vocab[token] for token in eng_lng_tokenizer(sentence.rstrip("\n"))], dtype=torch.long)
        train_data.append((2, text_tensor))
for i in range(len(ag_test_list_sub)):
    sentences = sent_tokenize(ag_test_list_sub[i][1])
    for sentence in sentences:
        text_tensor = torch.tensor([eng_lng_vocab[token] for token in eng_lng_tokenizer(sentence.rstrip("\n"))], dtype=torch.long)
        test_data.append((2, text_tensor))


In [35]:
BATCH_SIZE = 1

# Split dataset
num_class = 2
num_train = int(len(train_data) * 0.95)
split_train_, split_valid_ = \
    random_split(train_data, [num_train, len(train_data) - num_train])

def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text_tensor) in batch:
        label_list.append(_label - 1)
        new_tensor = torch.cat([torch.tensor([BOS_IDX]), _text_tensor, torch.tensor([EOS_IDX])], dim=0)
        text_list.append(new_tensor)
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, padding_value=PAD_IDX)
    return label_list.to(device), text_list.to(device)

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

print("Train size:", len(train_dataloader.dataset))
print("Val size:", len(valid_dataloader.dataset))
print("Test size:", len(test_dataloader.dataset))

Train size: 6389
Val size: 337
Test size: 709


### Model

In [36]:
class TextStyleGenerator(nn.Module):
    def __init__(self, num_decoder_layers: int,
                 emb_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(TextStyleGenerator, self).__init__()
        
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)
        
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self, memory: Tensor, trg: Tensor, tgt_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

### Train

In [37]:
import time

def train(model, dataloader, crit1, crit2):
    transformer.eval()
    
    running_loss = 0.0
    for idx, (label, text) in enumerate(dataloader):
        print(idx)
        optimizer.zero_grad()
        
        src = text
        tgt_input = text[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
        # Encode first
        memory = transformer.encode(src, src_mask)
        
        # Train decoder
        logits = model(memory, tgt_input, tgt_mask, tgt_padding_mask, src_padding_mask)
        
        # Calculate loss
        tgt_out = text[1:,:]
        loss = crit1(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
#         if idx % 10 == 0:
#         val_loss = evaluate(text_styler, valid_dataloader, crit1)
        val_loss = 0
        print((f"Epoch: {epoch}, Train loss: {running_loss:.3f}, Val loss: {val_loss:.3f}"))

        running_loss = 0.0
        model.train()

def evaluate(model, dataloader, crit1):
    model.eval()
    losses = 0
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            src = text
            tgt_input = text[:-1, :]
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
            # Encode first
            memory = transformer.encode(src, src_mask)

            # Train decoder
            logits = model(memory, tgt_input, tgt_mask, tgt_padding_mask, src_padding_mask)

            # Calculate loss
            tgt_out = text[1:,:]
            loss = crit1(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            losses += loss.item()

        return losses / len(dataloader)

In [38]:
temp_data = train_data[:1]
temp_dataloader = DataLoader(temp_data, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [39]:
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_DECODER_LAYERS = 3

NUM_EPOCHS = 5


text_styler = TextStyleGenerator(NUM_DECODER_LAYERS,
                                 EMB_SIZE, ENG_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in text_styler.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

text_styler = text_styler.to(device)

crit1 = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
crit2 = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    text_styler.parameters(), lr=0.0005, betas=(0.9, 0.99), eps=1e-8
)

PATH_FINAL_TMP = './final_model_tmp.pt'
for epoch in range(1, NUM_EPOCHS+1):
    train_loss = train(text_styler, temp_dataloader, crit1, crit2)
    torch.save(text_styler, PATH_FINAL_TMP)

0
Epoch: 1, Train loss: 13.742, Val loss: 0.000
0
Epoch: 2, Train loss: 13.472, Val loss: 0.000
0
Epoch: 3, Train loss: 13.069, Val loss: 0.000
0
Epoch: 4, Train loss: 12.676, Val loss: 0.000
0
Epoch: 5, Train loss: 12.276, Val loss: 0.000


### Test

In [40]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    
    transformer.eval()
    memory = transformer.encode(src, src_mask)

    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def transfer(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [41]:
ag_train_list_sub[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [42]:
transfer(text_styler, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", eng_lng_vocab, eng_lng_vocab, eng_lng_tokenizer)

' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'

In [43]:
PATH_FINAL = './final_model.pt'

torch.save(text_styler, PATH_FINAL)